In [7]:
from neo4j.v1 import GraphDatabase, basic_auth
import itertools
from joblib import Parallel, delayed
import multiprocessing
import time

In [2]:
num_cores = multiprocessing.cpu_count()

In [3]:
driver = GraphDatabase.driver("bolt://localhost", auth=basic_auth("neo4j", "dmkm1234"))

/usr/local/lib/python2.7/dist-packages/neo4j/v1/session.py:94: UserWarning: Bolt over TLS is only available in Python 2.7.9+ and Python 3.3+ so communications are not secure
  warn("Bolt over TLS is only available in Python 2.7.9+ and Python 3.3+ "


In [4]:
session = driver.session()

In [23]:
t = time.time()
result = session.run("MATCH (ar:Articles)-[:isArticle]-(s:Signatures)-[:isAuthor]-(au:Authors) RETURN (ar.article_id) as article_id, collect(au.author_id) as authorlist limit 10")
for record in result:
    for pair in list(itertools.product(record['authorlist'],record['authorlist'])):
        query = session.run("MATCH (a1:Authors), (a2:Authors) WHERE a1.author_id = %s and a2.author_id = %s MERGE (a1)-[r:isCoauthor]-(a2)" % (pair[0], pair[1]))
elapsed = time.time() - t
print(elapsed)

SyntaxError: invalid syntax (<ipython-input-23-32d717acbf12>, line 4)

In [ ]:
session.close()

In [13]:
from math import sqrt
[sqrt(i ** 2) for i in range(10)]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]